In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt
from tensorflow.keras.applications import ResNet50

In [2]:
IMG_SIZE = 256
BATCH_SIZE = 32
EPOCHS = 10
CHANNELS = 3

In [3]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "tomatoImages",
    shuffle = True,
    image_size = IMG_SIZE,
    batch_size = BATCH_SIZE 
)

Found 2995 files belonging to 2 classes.


In [4]:
names  = dataset.class_names
print(names)

['Tomato__Target_Spot', 'Tomato_healthy']


In [5]:
# def test_train_spilit(dataset, train_size = 0.8, test_size = 0.1, val_size = 0.1, shuffle = True, shuffle_size = 10000):
#     ds_size = len(dataset)

#     if shuffle:
#         dataset = dataset.shuffle(shuffle_size, seed = 12)

#     train_ds = dataset.take(int(ds_size*train_size))
#     val_ds = dataset.skip(int(ds_size*train_size)).take(int(ds_size*val_size))
#     test_ds = dataset.skip(int(ds_size*train_size)).skip(int(ds_size*val_size))
                
#     return train_ds, val_ds, test_ds

In [10]:
# this is to make the result closer to the "Morgh" project.
# otherwise the 'test_train_spilit' function can be used.
def split_dataset_by_count(dataset, train_count , val_count , test_count, shuffle_size=10000, seed=12):
    """
    Shuffles, subsets, and splits a tf.data.Dataset by absolute counts.
    """
    dataset = dataset.shuffle(shuffle_size, seed=seed)
    
    dataset = dataset.unbatch()
    
    total_count = train_count + val_count + test_count
    subset_ds = dataset.take(total_count)
    
    train_ds = subset_ds.take(train_count)
    val_ds = subset_ds.skip(train_count).take(val_count)
    test_ds = subset_ds.skip(train_count + val_count).take(test_count)
    
    return train_ds, val_ds, test_ds

In [11]:
train_ds, val_ds, test_ds = split_dataset_by_count(dataset, 
                                                   train_count=200, 
                                                   val_count=20, 
                                                   test_count=40)

In [12]:
# print(len(train_ds), len(val_ds), len(test_ds))

TypeError: The dataset length is unknown.

In [13]:
train_ds = train_ds.batch(BATCH_SIZE).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

In [14]:
resize = tf.keras.Sequential([
    layers.Resizing(IMG_SIZE,IMG_SIZE),
])
# rescaling is not needed due to resnet50 preprocessing function

In [15]:
data_augmentation =  tf.keras.Sequential([
    layers.RandomFlip("horizantal"),
    layers.RandomRotation(0.2),
])

In [16]:
INPUT_SHAPE = (IMG_SIZE, IMG_SIZE, CHANNELS)
NUM_CLASSES = len(names)

resnet_model = tf.keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=INPUT_SHAPE
)

resnet_model.trainable = False

preprocess_input = tf.keras.applications.resnet50.preprocess_input

In [17]:
model = models.Sequential([
    resize, 
    data_augmentation,
    layers.Lambda(preprocess_input, input_shape=INPUT_SHAPE),
    resnet_model,
    layers.GlobalAveragePooling2D(), 

    layers.Dense(128, activation='relu'), 
    layers.Dropout(0.2), 
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.2), 

    layers.Dense(NUM_CLASSES, activation='softmax') 
])

D:\azin\tomato\code\venv\Lib\site-packages\keras\src\layers\core\lambda_layer.py:65: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [18]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics = ['accuracy']
)

In [19]:
history = model.fit(
    train_ds,
    epochs =  EPOCHS,
    batch_size = BATCH_SIZE,
    verbose = 1,
    validation_data = val_ds
)

Epoch 1/10

      7/Unknown 25s 2s/step - accuracy: 0.5987 - loss: 0.8056

D:\azin\tomato\code\venv\Lib\site-packages\keras\src\trainers\epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


7/7 ━━━━━━━━━━━━━━━━━━━━ 31s 3s/step - accuracy: 0.6350 - loss: 0.7060 - val_accuracy: 0.7500 - val_loss: 0.4718
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.7050 - loss: 0.5906 - val_accuracy: 0.9500 - val_loss: 0.2244
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.7650 - loss: 0.4407 - val_accuracy: 0.9000 - val_loss: 0.2673
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.9000 - loss: 0.2749 - val_accuracy: 1.0000 - val_loss: 0.1145
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.9050 - loss: 0.2278 - val_accuracy: 1.0000 - val_loss: 0.1193
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.9300 - loss: 0.1288 - val_accuracy: 1.0000 - val_loss: 0.0658
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.9500 - loss: 0.1157 - val_accuracy: 1.0000 - val_loss: 0.0884
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 14s 2s/step - accuracy: 0.9600 - loss: 0.1031 - val_accuracy: 1.0000 - val_loss: 0.0240
Epoch 9/10
7/7 ━━━━

In [20]:
resnet_model.trainable = True

for layer in resnet_model.layers[:-30]: # Fine-tune top 30 layers, adjust as needed
    layer.trainable = False

In [21]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [22]:
fine_tune_epochs = 10
total_epochs = 10 + fine_tune_epochs

In [23]:
history_fine_tune = model.fit(
    train_ds,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1], 
    validation_data=val_ds
)

Epoch 10/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 39s 3s/step - accuracy: 0.8150 - loss: 0.4038 - val_accuracy: 1.0000 - val_loss: 0.0289
Epoch 11/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.8950 - loss: 0.2357 - val_accuracy: 1.0000 - val_loss: 0.0507
Epoch 12/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step - accuracy: 0.9600 - loss: 0.1063 - val_accuracy: 1.0000 - val_loss: 0.0818
Epoch 13/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.9700 - loss: 0.0881 - val_accuracy: 0.9000 - val_loss: 0.1229
Epoch 14/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.9800 - loss: 0.0551 - val_accuracy: 0.9000 - val_loss: 0.1527
Epoch 15/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 17s 2s/step - accuracy: 0.9700 - loss: 0.0759 - val_accuracy: 0.9000 - val_loss: 0.1556
Epoch 16/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.9950 - loss: 0.0336 - val_accuracy: 0.9000 - val_loss: 0.1481
Epoch 17/20
7/7 ━━━━━━━━━━━━━━━━━━━━ 18s 3s/step - accuracy: 0.9900 - loss: 0.0476 - val_accuracy: 0.9000 - val_loss: 0.1233


In [24]:
scores  = model.evaluate(test_ds) 

2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 614ms/step - accuracy: 0.8250 - loss: 0.2942
